# SYMFLUENCE Tutorial 04c — Sagehen Creek Workshop (Coupled SUMMA-MODFLOW Groundwater)

## Introduction

This workshop notebook demonstrates how to set up a **coupled SUMMA-MODFLOW groundwater model** for Sagehen Creek near Truckee, California. The workflow couples a land surface model (SUMMA) with a groundwater model (MODFLOW 6) to simulate both surface and subsurface flow processes.

**Sagehen Creek** (USGS 10343500) is a well-known USGS research watershed in the Sierra Nevada. It is a ~27 km² snow-dominated headwater catchment (elevation ~1,930–2,660 m) with decades of hydrological observations — ideal for demonstrating coupled surface-groundwater modeling.

### Coupling Architecture

```
SUMMA (Land Surface)        MODFLOW 6 (Groundwater)
┌──────────────────┐        ┌──────────────────────┐
│  Snow processes   │        │  Saturated zone flow  │
│  Canopy processes │        │  (hydraulic cond. K)  │
│  Soil column      │───────>│  Recharge (from SUMMA)│
│  Surface runoff   │ rech.  │  Drain discharge      │
└────────┬─────────┘        └──────────┬───────────┘
         │                              │
         │ surface runoff (m³/s)        │ baseflow (m³/s)
         │                              │
         └──────────┐  ┌───────────────┘
                    ▼  ▼
            Combined Streamflow
            (compared to USGS obs)
```

The workflow includes:

1. **Configuration** — Set up a coupled SUMMA-MODFLOW model for Sagehen Creek
2. **Domain Definition** — Delineate the watershed using TauDEM
3. **Data Acquisition** — Fetch AORC forcing data and USGS streamflow observations
4. **Model Execution** — Run the coupled SUMMA→MODFLOW pipeline and visualize coupling
5. **Evaluation & Calibration** — Assess performance and calibrate 17 parameters (14 SUMMA + 3 MODFLOW)

**Launch this notebook from the CLI:**
```bash
symfluence example launch 4c
```

In [ ]:
# Environment verification
import sys
import warnings
from pathlib import Path

# Suppress experimental module warnings for cleaner output
warnings.filterwarnings('ignore', message='.*is an EXPERIMENTAL module.*')
warnings.filterwarnings('ignore', message='.*import failed.*')

print(f"Python executable: {sys.executable}")

# Verify SYMFLUENCE is available
try:
    import symfluence
    print(f"SYMFLUENCE version: {symfluence.__version__}")
    print(f"SYMFLUENCE location: {Path(symfluence.__file__).parent}")
except ImportError:
    print("ERROR: SYMFLUENCE not found. Please activate the symfluence environment.")
    sys.exit(1)

In [ ]:
# Fix working directory if running from .ipynb_checkpoints
import os
from pathlib import Path

current_dir = Path.cwd()
print(f"Current directory: {current_dir}")

# If we're in .ipynb_checkpoints, move up to parent directory
if '.ipynb_checkpoints' in str(current_dir):
    correct_dir = current_dir.parent
    os.chdir(correct_dir)
    print(f"Changed to: {Path.cwd()}")
else:
    print("Working directory is correct")

# Verify we're in the workshop notebooks directory
expected_notebook = Path.cwd() / '04c_sagehen_creek_workshop.ipynb'
if not expected_notebook.exists():
    print(f"WARNING: Expected notebook not found at {expected_notebook}")
else:
    print("Notebook location verified")

## Step 1 — Configuration

Create a configuration for the Sagehen Creek coupled SUMMA-MODFLOW model. Key settings:
- **Model**: `SUMMA` — land surface hydrology (snow, canopy, soil column, surface runoff)
- **Groundwater**: `MODFLOW` — MODFLOW 6 groundwater flow (set via `GROUNDWATER_MODEL`)
- **Coupling**: Sequential — SUMMA runs first, recharge feeds MODFLOW, flows are combined
- **Routing**: MODFLOW drain (no separate routing model needed)
- **Forcing**: AORC (Analysis of Record for Calibration) — 1km hourly gridded data
- **Observations**: USGS streamflow from station 10343500
- **Period**: 5 years (2016-2020) with 1-year spinup
- **Calibration parameters**: 14 total (11 SUMMA + 3 MODFLOW)

### MODFLOW Domain Setup

The lumped MODFLOW cell represents the fractured granite aquifer beneath Sagehen Creek:
- **TOP** = 2300 m, **BOT** = 2100 m (200 m aquifer thickness)
- **Drain elevation** = 2200 m (mid-aquifer, representing stream-aquifer connection)
- **K** = 5.0 m/d initial hydraulic conductivity
- **SY** = 0.05 initial specific yield (fractured granite — lower than alluvial aquifers)
- **Cell size** ≈ 5436 m (auto-computed from catchment area ~29.5 km²)

### Aquifer Time Constant

The lumped MODFLOW cell acts as a linear reservoir with time constant τ = SY × A / C_drain.
With SY=0.05, A=29.5 km², C=15,000 m²/d: τ ≈ 98 days — allowing seasonal variability while
maintaining the smoothing characteristic of groundwater storage.

In [ ]:
# Step 1 — Create coupled SUMMA-MODFLOW configuration

from pathlib import Path
from symfluence import SYMFLUENCE
from symfluence.core.config.models import SymfluenceConfig
import os

# === Sagehen Creek Coupled Configuration ===

# Ensure we're working from the correct directory
current_dir = Path.cwd()
print(f"INITIAL working directory: {current_dir}")

if '.ipynb_checkpoints' in str(current_dir):
    current_dir = current_dir.parent
    os.chdir(current_dir)
    print(f"CHANGED to: {Path.cwd()}")
else:
    print("Working directory is correct")

# Derive paths dynamically from repo structure
# Notebooks live at: <repo>/examples/04_workshop_notebooks/
repo_root = current_dir.parent.parent
data_dir = repo_root.parent / 'SYMFLUENCE_data'

print(f"Notebook directory: {current_dir}")
print(f"Repo root (CODE_DIR): {repo_root}")
print(f"Data directory (DATA_DIR): {data_dir}")

config = SymfluenceConfig.from_minimal(
    # ============================================================================
    # BASIC IDENTIFICATION
    # ============================================================================
    domain_name='Sagehen_Creek_near_Truckee',
    experiment_id='workshop_run_3',

    # ============================================================================
    # PATHS — use 'default' to auto-resolve from DATA_DIR
    # ============================================================================
    SYMFLUENCE_DATA_DIR=str(data_dir),
    SYMFLUENCE_CODE_DIR=str(repo_root),
    TAUDEM_DIR='default',
    SUMMA_INSTALL_PATH='default',

    # ============================================================================
    # COUPLED MODEL SETUP
    # ============================================================================
    model='SUMMA',                               # SUMMA for land surface processes
    GROUNDWATER_MODEL='MODFLOW',                 # MODFLOW 6 for groundwater
    COUPLING_MODE='sequential',                  # Sequential coupling (SUMMA -> MODFLOW)
    routing_model='none',                        # MODFLOW drain provides baseflow

    # ============================================================================
    # MODFLOW DOMAIN PARAMETERS
    # ============================================================================
    MODFLOW_K=5.0,                               # Hydraulic conductivity (m/d)
    MODFLOW_SY=0.05,                             # Specific yield — fractured granite
    MODFLOW_TOP=2300,                            # Top of aquifer (m)
    MODFLOW_BOT=2100,                            # Bottom of aquifer (m)
    MODFLOW_DRAIN_ELEVATION=2200,                # Drain elevation (m)
    MODFLOW_DRAIN_CONDUCTANCE=1.5e4,             # Drain conductance (m2/d)

    # ============================================================================
    # SUMMA DECISION OVERRIDES
    # ============================================================================
    SUMMA_DECISION_OPTIONS={'hc_profile': ['pow_prof']},  # K decreases with depth

    # ============================================================================
    # SIMULATION PERIOD (5 years: 2016-2020)
    # ============================================================================
    time_start='2016-01-01 01:00',
    time_end='2020-12-31 23:00',

    # Time period definitions
    spinup_period='2016-01-01, 2016-12-31',        # Year 1: Model spinup
    calibration_period='2017-01-01, 2019-12-31',   # Years 2-4: Parameter calibration
    evaluation_period='2020-01-01, 2020-12-31',    # Year 5: Model evaluation

    # ============================================================================
    # SPATIAL DOMAIN (Sagehen Creek near Truckee, CA)
    # ============================================================================
    # USGS station 10343500: 39.4313N, 120.2383W
    pour_point_coords='39.4313/-120.2383',
    bounding_box_coords='39.48/-120.30/39.38/-120.18',

    # Domain discretization
    definition_method='lumped',
    discretization='GRUs',
    lumped_watershed_method='TauDEM',

    # ============================================================================
    # DATA SOURCES & FORCING
    # ============================================================================
    data_access='cloud',
    forcing_dataset='AORC',
    forcing_measurement_height=10,
    dem_source='copernicus',
    download_dem=True,

    # ============================================================================
    # STREAMFLOW OBSERVATIONS
    # ============================================================================
    station_id='10343500',
    streamflow_data_provider='USGS',
    download_usgs_data=True,

    # ============================================================================
    # CALIBRATION PARAMETERS (14 SUMMA + 3 MODFLOW = 17 total)
    # ============================================================================
    OPTIMIZATION_METHODS=['iteration'],

    # SUMMA land surface parameters (12 local + 2 basin = 14)
    params_to_calibrate='albedoMax,albedoMinWinter,newSnowDenMin,Fcapil,k_soil,theta_sat,critSoilWilting,theta_res,f_impede,tempCritRain,fieldCapacity,qSurfScale',
    BASIN_PARAMS_TO_CALIBRATE='routingGammaShape,routingGammaScale',

    # MODFLOW groundwater parameters (3)
    MODFLOW_PARAMS_TO_CALIBRATE='K,SY,DRAIN_CONDUCTANCE',

    # Parameter bound overrides
    PARAMETER_BOUNDS={
        'K': [0.1, 100.0],
        'SY': [0.005, 0.25],
        'fieldCapacity': [0.05, 0.45],
        'critSoilWilting': [0.01, 0.40],
        'albedoMax': [0.50, 0.95],
        'albedoMinWinter': [0.45, 0.85],
        'theta_sat': [0.35, 0.85],
        'theta_res': [0.01, 0.15],
        'routingGammaShape': [1.0, 5.0],
        'DRAIN_CONDUCTANCE': [1000.0, 500000.0],
    },

    # Optimization configuration
    optimization_target='streamflow',
    optimization_algorithm='DDS',
    optimization_metric='KGE',
    calibration_timestep='daily',
    iterations=300,
)

# Verify
print(f"\nConfig verification:")
print(f"  Model: {config.model.hydrological_model}")
print(f"  Groundwater: {config.model.groundwater_model}")
print(f"  Data dir: {config.system.data_dir}")
print(f"  Optimization methods: {config.optimization.methods}")

# Save configuration
config_path = Path('./config_sagehen_creek_coupled.yaml')
config_dict = config.to_dict(flatten=True)
import yaml
with open(config_path, 'w') as f:
    yaml.dump(config_dict, f, default_flow_style=False, sort_keys=False)
print(f"\nConfiguration saved to: {config_path}")

# Initialize SYMFLUENCE
sf = SYMFLUENCE(config)
project_dir = sf.managers['project'].setup_project()
pour_point_path = sf.managers['project'].create_pour_point()

print(f"\nProject structure created at: {project_dir}")
print(f"Pour point shapefile: {pour_point_path}")
print("=" * 80)

## Step 2 — Domain Definition

Delineate the Sagehen Creek watershed using TauDEM and create a single lumped HRU.

### Step 2a — Geospatial Attribute Acquisition

Acquire elevation, land cover, and soil data from cloud sources.

In [ ]:
# Step 2a — Acquire geospatial attributes from cloud
sf.managers['data'].acquire_attributes()
print("Attribute acquisition complete")

### Step 2b — Watershed Delineation

Delineate the watershed boundary from the pour point using TauDEM.

In [ ]:
# Step 2b — Watershed delineation
watershed_path = sf.managers['domain'].define_domain()
print(f"Watershed delineation complete")
print(f"Watershed file: {watershed_path}")

### Step 2c — Domain Discretization

Create a single lumped HRU for the watershed.

In [ ]:
# Step 2c — Discretization (single lumped HRU)
hru_path = sf.managers['domain'].discretize_domain()
print("Domain discretization complete")
print(f"HRU file: {hru_path}")

### Step 2d — Visualization

Visualize the delineated Sagehen Creek watershed and pour point.

In [ ]:
# Step 2d — Basin visualization

import geopandas as gpd
import matplotlib.pyplot as plt

# Load spatial data
basin_path = project_dir / 'shapefiles' / 'river_basins' / f"{config.domain.name}_riverBasins_lumped.shp"
hru_file = project_dir / 'shapefiles' / 'catchment' / 'lumped' / config.domain.experiment_id / f"{config.domain.name}_HRUs_GRUs.shp"

watershed_gdf = gpd.read_file(str(basin_path))
hru_gdf = gpd.read_file(str(hru_file))
pour_point_gdf = gpd.read_file(pour_point_path)

# Calculate area (UTM Zone 10N for northern California)
watershed_proj = watershed_gdf.to_crs('EPSG:32610')
area_km2 = watershed_proj.geometry.area.sum() / 1e6

# Plot
fig, ax = plt.subplots(1, 1, figsize=(10, 10))
watershed_gdf.boundary.plot(ax=ax, color='blue', linewidth=2, label='Watershed')
hru_gdf.plot(ax=ax, facecolor='lightblue', edgecolor='blue', alpha=0.3)
pour_point_gdf.plot(ax=ax, color='red', markersize=150, marker='*',
                    label=f'Pour Point (USGS {config.evaluation.streamflow.station_id})')

ax.set_title(f"Sagehen Creek near Truckee\nArea: {area_km2:.0f} km\u00b2",
             fontweight='bold', fontsize=14)
ax.legend(loc='upper right')
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')
plt.tight_layout()
plt.show()

print(f"Watershed area: {area_km2:.0f} km\u00b2")
print(f"Number of HRUs: {len(hru_gdf)} (lumped)")

## Step 3 — Data Acquisition and Preprocessing

Fetch forcing data (AORC) and streamflow observations (USGS) from cloud sources.

### Step 3a — USGS Streamflow Observations

Download and process USGS streamflow data for station 10343500 (Sagehen Creek near Truckee, CA).

In [ ]:
# Step 3a — Download and process USGS streamflow data
sf.managers['data'].process_observed_data()

print("USGS streamflow data acquisition complete")

### Step 3b — AORC Meteorological Forcing

Download AORC forcing data from NOAA's cloud archive (AWS S3). AORC provides:
- 1 km spatial resolution
- Hourly temporal resolution
- Complete forcing variables: precipitation, temperature, humidity, wind, radiation, pressure

In [ ]:
# Step 3b — Acquire AORC forcing data from cloud
sf.managers['data'].acquire_forcings()
print("AORC forcing acquisition complete")

### Step 3c — Model-Agnostic Preprocessing

Standardize forcing data: variable names, units, and spatial averaging over the watershed.

In [ ]:
# Step 3c — Model-agnostic preprocessing
sf.managers['data'].run_model_agnostic_preprocessing()
print("Model-agnostic preprocessing complete")

## Step 4 — Model Configuration and Execution

Configure the coupled SUMMA-MODFLOW pipeline and run the simulation. This step:
1. Creates **SUMMA settings** (forcing lists, parameters, cold state)
2. Creates **MODFLOW settings** (gwf.nam, gwf.dis, gwf.npf, gwf.sto, gwf.drn, gwf.rch, etc.)
3. Runs the coupled pipeline: SUMMA → extract recharge → MODFLOW → combine flows

In [ ]:
# Step 4a — Model preprocessing (creates BOTH SUMMA and MODFLOW settings)
sf.managers['model'].preprocess_models()
print("SUMMA + MODFLOW configuration complete")

In [ ]:
# Step 4b — Run coupled model
print(f"Running coupled pipeline: SUMMA (land surface) -> MODFLOW (groundwater)...")
print(f"Simulation period: {config.domain.time_start} to {config.domain.time_end}")
print(f"Coupling mode: sequential")
print(f"Workflow: HYDROLOGICAL_MODEL={config.model.hydrological_model}, GROUNDWATER_MODEL={config.model.groundwater_model}")
sf.managers['model'].run_models()
print("Coupled SUMMA-MODFLOW simulation complete")

In [ ]:
# Step 4c — Verify output files exist
sim_dir = project_dir / 'simulations' / config.domain.experiment_id

# Check SUMMA output
summa_dir = sim_dir / 'SUMMA'
summa_files = list(summa_dir.glob('*.nc')) if summa_dir.exists() else []
print(f"SUMMA output directory: {summa_dir}")
print(f"  NetCDF files found: {len(summa_files)}")
for f in summa_files:
    print(f"    {f.name} ({f.stat().st_size / 1e6:.1f} MB)")

# Check MODFLOW output
modflow_dir = sim_dir / 'MODFLOW'
modflow_files = list(modflow_dir.glob('*')) if modflow_dir.exists() else []
print(f"\nMODFLOW output directory: {modflow_dir}")
for f in sorted(modflow_files):
    if f.is_file():
        print(f"    {f.name} ({f.stat().st_size / 1e3:.1f} KB)")

### Step 4d — Coupling Visualization

Visualize the coupling between SUMMA and MODFLOW:
- **Recharge**: SUMMA soil drainage → MODFLOW recharge input
- **Drain discharge**: MODFLOW baseflow contribution
- **Flow partitioning**: Surface runoff vs. baseflow

In [ ]:
# Step 4d — Coupling visualization

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd

from symfluence.models.modflow.coupling import SUMMAToMODFLOWCoupler
from symfluence.models.modflow.extractor import MODFLOWResultExtractor

# Get basin area for unit conversions
basin_path = project_dir / 'shapefiles' / 'river_basins' / f"{config.domain.name}_riverBasins_lumped.shp"
watershed_gdf = gpd.read_file(str(basin_path))
watershed_proj = watershed_gdf.to_crs('EPSG:32610')  # UTM Zone 10N
basin_area_m2 = watershed_proj.geometry.area.sum()
basin_area_km2 = basin_area_m2 / 1e6
print(f"Basin area: {basin_area_km2:.2f} km\u00b2")

# Initialize coupler and extractor
coupler = SUMMAToMODFLOWCoupler(config.to_dict(flatten=True))
extractor = MODFLOWResultExtractor()

summa_dir = project_dir / 'simulations' / config.domain.experiment_id / 'SUMMA'
modflow_dir = project_dir / 'simulations' / config.domain.experiment_id / 'MODFLOW'

# Extract recharge (SUMMA -> MODFLOW)
recharge = coupler.extract_recharge_from_summa(summa_dir, variable='scalarSoilDrainage')
print(f"Recharge: {len(recharge)} timesteps, mean = {recharge.mean():.4f} m/d")

# Extract fast runoff from SUMMA (total routed runoff minus soil drainage)
# scalarSurfaceRunoff is only Hortonian overland flow (near-zero for most catchments).
# The correct fast-flow component is: averageRoutedRunoff - scalarSoilDrainage
fast_runoff = coupler.extract_fast_runoff(summa_dir)
print(f"Fast runoff: {len(fast_runoff)} timesteps, mean = {fast_runoff.mean():.6e} m/s")

# Extract drain discharge from MODFLOW
drain_discharge = extractor.extract_variable(
    modflow_dir, variable_type='drain_discharge',
    start_date=str(config.domain.time_start)[:10],
)
print(f"Drain discharge: {len(drain_discharge)} timesteps, mean = {drain_discharge.mean():.2f} m3/d")

# Combine flows (fast runoff is in m/s, same units as surface_runoff was)
total_flow = coupler.combine_flows(fast_runoff, drain_discharge, basin_area_m2)

# Convert components for plotting
fast_m3s = fast_runoff * basin_area_m2
fast_m3s.index = fast_m3s.index.normalize()
baseflow_m3s = drain_discharge / 86400.0
baseflow_m3s.index = baseflow_m3s.index.normalize()

# === Coupling plots ===
fig, axes = plt.subplots(3, 1, figsize=(14, 12), sharex=True)

# Panel 1: Recharge time series (SUMMA -> MODFLOW)
axes[0].fill_between(recharge.index, recharge.values, alpha=0.6, color='steelblue')
axes[0].set_ylabel('Recharge (m/d)')
axes[0].set_title('SUMMA Soil Drainage \u2192 MODFLOW Recharge', fontweight='bold')
axes[0].grid(True, alpha=0.3)

# Panel 2: Flow components
common_idx = fast_m3s.index.intersection(baseflow_m3s.index)
if len(common_idx) > 0:
    fast_plot = fast_m3s.loc[common_idx]
    base_plot = baseflow_m3s.loc[common_idx]
    axes[1].fill_between(common_idx, fast_plot.values, alpha=0.6, color='coral', label='Fast runoff (routed - drainage)')
    axes[1].fill_between(common_idx, fast_plot.values, fast_plot.values + base_plot.values,
                         alpha=0.6, color='steelblue', label='MODFLOW baseflow')
    axes[1].set_ylabel('Discharge (m\u00b3/s)')
    axes[1].set_title('Flow Component Breakdown', fontweight='bold')
    axes[1].legend(loc='upper right')
    axes[1].grid(True, alpha=0.3)

# Panel 3: Combined streamflow
axes[2].plot(total_flow.index, total_flow.values, 'k-', linewidth=1.5, label='Combined streamflow')
axes[2].set_ylabel('Discharge (m\u00b3/s)')
axes[2].set_xlabel('Date')
axes[2].set_title('Combined Streamflow (Fast Runoff + Baseflow)', fontweight='bold')
axes[2].legend(loc='upper right')
axes[2].grid(True, alpha=0.3)

plt.suptitle('Sagehen Creek \u2014 SUMMA-MODFLOW Coupling Overview', fontsize=14, fontweight='bold', y=1.01)
plt.tight_layout()
plt.show()

# Print baseflow index
if len(common_idx) > 0:
    total_mean = (fast_plot + base_plot).mean()
    bfi = base_plot.mean() / total_mean if total_mean > 0 else 0
    print(f"\nBaseflow Index (BFI): {bfi:.2f}")
    print(f"Mean fast runoff: {fast_plot.mean():.4f} m\u00b3/s")
    print(f"Mean baseflow: {base_plot.mean():.4f} m\u00b3/s")
    print(f"Mean total flow: {total_mean:.4f} m\u00b3/s")

## Step 5 — Evaluation and Calibration

Compare simulated combined streamflow against USGS observations and calibrate parameters.

### Step 5a — Uncalibrated Evaluation

Evaluate the uncalibrated coupled model against USGS observations at station 10343500.

In [ ]:
# Step 5a — Uncalibrated evaluation

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr

# Load observed streamflow
obs_path = project_dir / 'observations' / 'streamflow' / 'preprocessed' / f"{config.domain.name}_streamflow_processed.csv"
obs_df = pd.read_csv(obs_path, parse_dates=['datetime'])
obs_df.set_index('datetime', inplace=True)

# Use the combined flow from Step 4d (already computed)
sim_daily = total_flow.copy()
sim_daily.name = 'discharge_sim'

# Exclude spinup period
spinup_end = pd.to_datetime(config.domain.spinup_period.split(',')[1].strip())
print(f"Excluding spinup period up to: {spinup_end}")

# Resample observations to daily for comparison with MODFLOW daily output
obs_daily = obs_df['discharge_cms'].resample('D').mean().dropna()

# Align observed and simulated
common_idx = obs_daily.index.intersection(sim_daily.index)
common_idx = common_idx[common_idx > spinup_end]

obs_valid = obs_daily.loc[common_idx]
sim_valid = sim_daily.loc[common_idx]

print(f"Evaluation period: {obs_valid.index[0]} to {obs_valid.index[-1]}")
print(f"Number of timesteps: {len(obs_valid)}")

# Calculate evaluation metrics
def nse(obs, sim):
    return float(1 - np.sum((obs - sim)**2) / np.sum((obs - obs.mean())**2))

def kge(obs, sim):
    r = obs.corr(sim)
    alpha = sim.std() / obs.std()
    beta = sim.mean() / obs.mean()
    return float(1 - np.sqrt((r-1)**2 + (alpha-1)**2 + (beta-1)**2))

def pbias(obs, sim):
    return float(100 * (sim.sum() - obs.sum()) / obs.sum())

uncal_metrics = {
    'NSE': round(nse(obs_valid, sim_valid), 3),
    'KGE': round(kge(obs_valid, sim_valid), 3),
    'PBIAS': round(pbias(obs_valid, sim_valid), 1)
}

print(f"\nPerformance Metrics (Uncalibrated):")
for k, v in uncal_metrics.items():
    print(f"  {k}: {v}")

In [ ]:
# Visualization — standard evaluation plots + flow component breakdown

fig, axes = plt.subplots(3, 2, figsize=(14, 14))

# --- Row 1: Time series (top left) ---
axes[0, 0].plot(obs_valid.index, obs_valid.values, 'b-', label='Observed (USGS)', linewidth=1.2, alpha=0.7)
axes[0, 0].plot(sim_valid.index, sim_valid.values, 'r-', label='Simulated (SUMMA+MODFLOW)', linewidth=1.2, alpha=0.7)
axes[0, 0].set_ylabel('Discharge (m\u00b3/s)')
axes[0, 0].set_title('Streamflow Time Series')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)
axes[0, 0].text(0.02, 0.95, f"NSE: {uncal_metrics['NSE']}\nKGE: {uncal_metrics['KGE']}\nBias: {uncal_metrics['PBIAS']}%",
                transform=axes[0, 0].transAxes, verticalalignment='top',
                bbox=dict(facecolor='white', alpha=0.8), fontsize=9)

# --- Row 1: Scatter (top right) ---
axes[0, 1].scatter(obs_valid, sim_valid, alpha=0.5, s=10)
max_val = max(obs_valid.max(), sim_valid.max())
axes[0, 1].plot([0, max_val], [0, max_val], 'k--', alpha=0.5)
axes[0, 1].set_xlabel('Observed (m\u00b3/s)')
axes[0, 1].set_ylabel('Simulated (m\u00b3/s)')
axes[0, 1].set_title('Observed vs Simulated')
axes[0, 1].grid(True, alpha=0.3)

# --- Row 2: Monthly climatology (middle left) ---
monthly_obs = obs_valid.groupby(obs_valid.index.month).mean()
monthly_sim = sim_valid.groupby(sim_valid.index.month).mean()
month_names = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun',
               'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
axes[1, 0].plot(monthly_obs.index, monthly_obs.values, 'b-o', label='Observed', markersize=6)
axes[1, 0].plot(monthly_sim.index, monthly_sim.values, 'r-o', label='Simulated', markersize=6)
axes[1, 0].set_xticks(range(1, 13))
axes[1, 0].set_xticklabels(month_names)
axes[1, 0].set_ylabel('Mean Discharge (m\u00b3/s)')
axes[1, 0].set_title('Seasonal Flow Regime')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

# --- Row 2: Flow duration curve (middle right) ---
obs_sorted = obs_valid.sort_values(ascending=False)
sim_sorted = sim_valid.sort_values(ascending=False)
obs_ranks = np.arange(1., len(obs_sorted) + 1) / len(obs_sorted) * 100
sim_ranks = np.arange(1., len(sim_sorted) + 1) / len(sim_sorted) * 100
axes[1, 1].semilogy(obs_ranks, obs_sorted, 'b-', label='Observed', linewidth=2)
axes[1, 1].semilogy(sim_ranks, sim_sorted, 'r-', label='Simulated', linewidth=2)
axes[1, 1].set_xlabel('Exceedance Probability (%)')
axes[1, 1].set_ylabel('Discharge (m\u00b3/s)')
axes[1, 1].set_title('Flow Duration Curve')
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)

# --- Row 3: Flow component breakdown (bottom left) ---
comp_idx = fast_m3s.index.intersection(baseflow_m3s.index)
comp_idx = comp_idx[comp_idx > spinup_end]
if len(comp_idx) > 0:
    fast_eval = fast_m3s.loc[comp_idx]
    base_eval = baseflow_m3s.loc[comp_idx]
    axes[2, 0].fill_between(comp_idx, 0, fast_eval.values, alpha=0.6, color='coral', label='Fast runoff')
    axes[2, 0].fill_between(comp_idx, fast_eval.values, fast_eval.values + base_eval.values,
                            alpha=0.6, color='steelblue', label='MODFLOW baseflow')
    axes[2, 0].plot(obs_valid.index, obs_valid.values, 'k-', linewidth=1, alpha=0.7, label='Observed')
axes[2, 0].set_ylabel('Discharge (m\u00b3/s)')
axes[2, 0].set_xlabel('Date')
axes[2, 0].set_title('Flow Components vs Observed')
axes[2, 0].legend(loc='upper right')
axes[2, 0].grid(True, alpha=0.3)

# --- Row 3: Monthly baseflow fraction (bottom right) ---
if len(comp_idx) > 0:
    monthly_fast = fast_eval.groupby(fast_eval.index.month).mean()
    monthly_base = base_eval.groupby(base_eval.index.month).mean()
    monthly_total = monthly_fast + monthly_base

    bars1 = axes[2, 1].bar(range(1, 13), monthly_fast.values, color='coral', alpha=0.7, label='Fast runoff')
    bars2 = axes[2, 1].bar(range(1, 13), monthly_base.values, bottom=monthly_fast.values,
                           color='steelblue', alpha=0.7, label='Baseflow')
    axes[2, 1].set_xticks(range(1, 13))
    axes[2, 1].set_xticklabels(month_names)
    axes[2, 1].set_ylabel('Mean Discharge (m\u00b3/s)')
    axes[2, 1].set_title('Monthly Flow Partitioning')
    axes[2, 1].legend()
    axes[2, 1].grid(True, alpha=0.3, axis='y')

plt.suptitle('Sagehen Creek \u2014 Uncalibrated SUMMA+MODFLOW Evaluation', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

print("\nUncalibrated evaluation complete")

### Step 5b — Calibration

Calibrate 17 parameters jointly using DDS (Dynamically Dimensioned Search):
- **14 SUMMA parameters**: albedo, snow density, soil hydraulics, precipitation phase, surface runoff scaling, routing
- **3 MODFLOW parameters**: hydraulic conductivity (K), specific yield (SY), drain conductance

The optimizer jointly adjusts both SUMMA and MODFLOW parameters to maximize KGE.

In [ ]:
# Step 5b — Run calibration
print(f"Starting coupled calibration...")
print(f"Algorithm: {config.optimization.algorithm}")
print(f"Metric: {config.optimization.metric}")
print(f"Iterations: {config.optimization.iterations}")
print(f"Calibration period: {config.domain.calibration_period}")
print(f"Parameters: 14 SUMMA + 3 MODFLOW = 17 total")

results_file = sf.managers['optimization'].calibrate_model()
print(f"\nCalibration complete!")
print(f"Results file: {results_file}")

### View Calibration Progress

In [ ]:
# Load and display calibration results
import json as _json

# Find results matching our experiment_id
opt_base = project_dir / 'optimization' / 'COUPLED_GW'
exp_id = config.domain.experiment_id
run_dir = opt_base / f'dds_{exp_id}'

# Fallback: search for any dds run
if not run_dir.exists():
    run_dirs = sorted(opt_base.glob('dds_workshop_run_*')) if opt_base.exists() else []
    if not run_dirs:
        opt_base = project_dir / 'optimization' / 'SUMMA'
        run_dirs = sorted(opt_base.glob('dds_*'))
    run_dir = run_dirs[-1] if run_dirs else None

if run_dir and run_dir.exists():
    csv_files = list(run_dir.glob('*_parallel_iteration_results.csv'))
    bp_files = list(run_dir.glob('*_best_params.json'))
    fe_files = list(run_dir.glob('*_final_evaluation.json'))

    if csv_files:
        results_df = pd.read_csv(csv_files[0])
        # Compute running best
        results_df['best_score'] = results_df['score'].cummax()

        print(f"Calibration Progress ({run_dir.name}):")
        print(f"  Best {config.optimization.metric}: {results_df['score'].max():.4f}")
        print(f"  Initial {config.optimization.metric}: {results_df['score'].iloc[0]:.4f}")
        print(f"  Improvement: {results_df['score'].max() - results_df['score'].iloc[0]:.4f}")
        print(f"  Total iterations: {len(results_df)}")
        if 'crash_rate' in results_df.columns:
            print(f"  Crash rate: {results_df['crash_rate'].iloc[-1]:.1%}")

        # Plot calibration progress
        fig, ax = plt.subplots(figsize=(10, 5))
        ax.scatter(results_df['iteration'], results_df['score'],
                   alpha=0.3, s=10, color='gray', label='Per-iteration')
        ax.plot(results_df['iteration'], results_df['best_score'],
                'b-', linewidth=2, label='Running best')
        ax.set_xlabel('Iteration')
        ax.set_ylabel(f'Best {config.optimization.metric}')
        ax.set_title('Coupled SUMMA-MODFLOW Calibration Progress (DDS)')
        ax.legend()
        ax.grid(True, alpha=0.3)
        plt.tight_layout()
        plt.show()

        # Show calibrated parameters
        modflow_params = ['K', 'SY', 'DRAIN_CONDUCTANCE']
        best_idx = results_df['score'].idxmax()
        best_row = results_df.loc[best_idx]
        print(f"\nCalibrated MODFLOW parameters:")
        for p in modflow_params:
            if p in results_df.columns:
                print(f"  {p}: {best_row[p]:.4f}")

    # Show final evaluation if available
    if fe_files:
        with open(fe_files[0]) as f:
            fe_data = _json.load(f)
        print(f"\nFinal Evaluation:")
        cal_m = fe_data.get('calibration_metrics', {})
        eval_m = fe_data.get('evaluation_metrics', {})
        print(f"  Calibration  KGE={cal_m.get('KGE', 'N/A'):.3f}, NSE={cal_m.get('NSE', 'N/A'):.3f}, PBIAS={cal_m.get('PBIAS', 'N/A'):.1f}%")
        print(f"  Evaluation   KGE={eval_m.get('KGE', 'N/A'):.3f}, NSE={eval_m.get('NSE', 'N/A'):.3f}, PBIAS={eval_m.get('PBIAS', 'N/A'):.1f}%")
else:
    print("No calibration results found.")

### Step 5c — Post-Calibration Evaluation

Re-run the coupled model with calibrated parameters and evaluate performance improvement.

In [ ]:
# Step 5c — Post-calibration evaluation
# Re-extract combined streamflow from the calibrated final evaluation run

from symfluence.models.modflow.coupling import SUMMAToMODFLOWCoupler
from symfluence.models.modflow.extractor import MODFLOWResultExtractor

# Find the calibration output matching our experiment_id
opt_dir = project_dir / 'optimization' / 'COUPLED_GW'
exp_id = config.domain.experiment_id
best_dir = opt_dir / f'dds_{exp_id}'

# Fallback to most recent run if exact match not found
if not best_dir.exists():
    cal_dirs = sorted(opt_dir.glob('dds_workshop_run_*')) if opt_dir.exists() else []
    best_dir = cal_dirs[-1] if cal_dirs else None

if best_dir and best_dir.exists():
    print(f"Using calibration run: {best_dir.name}")

    # Use final_evaluation subdirectory (contains calibrated SUMMA + MODFLOW output)
    final_eval_dir = best_dir / 'final_evaluation'
    cal_summa_dir = final_eval_dir / 'SUMMA' if final_eval_dir.exists() else best_dir / 'SUMMA'
    cal_modflow_dir = final_eval_dir / 'MODFLOW' if final_eval_dir.exists() else best_dir / 'MODFLOW'

    print(f"  SUMMA output: {cal_summa_dir}")
    print(f"  MODFLOW output: {cal_modflow_dir}")

    if cal_summa_dir.exists() and cal_modflow_dir.exists():
        cal_coupler = SUMMAToMODFLOWCoupler(config.to_dict(flatten=True))
        cal_extractor = MODFLOWResultExtractor()

        cal_recharge = cal_coupler.extract_recharge_from_summa(cal_summa_dir)
        cal_fast = cal_coupler.extract_fast_runoff(cal_summa_dir)
        cal_drain = cal_extractor.extract_variable(
            cal_modflow_dir, variable_type='drain_discharge',
            start_date=str(config.domain.time_start)[:10],
        )
        cal_total = cal_coupler.combine_flows(cal_fast, cal_drain, basin_area_m2)

        # Evaluate calibrated model
        spinup_end_str = config.domain.spinup_period.split(',')[1].strip()
        spinup_end_local = pd.to_datetime(spinup_end_str)
        cal_common = obs_daily.index.intersection(cal_total.index)
        cal_common = cal_common[cal_common > spinup_end_local]

        obs_cal = obs_daily.loc[cal_common]
        sim_cal = cal_total.loc[cal_common]

        cal_metrics = {
            'NSE': round(nse(obs_cal, sim_cal), 3),
            'KGE': round(kge(obs_cal, sim_cal), 3),
            'PBIAS': round(pbias(obs_cal, sim_cal), 1)
        }

        print(f"\nPerformance Metrics (Calibrated):")
        for k, v in cal_metrics.items():
            print(f"  {k}: {v}")

        # Comparison plot
        fig, axes = plt.subplots(2, 2, figsize=(14, 10))

        # Time series
        axes[0, 0].plot(obs_cal.index, obs_cal.values, 'b-', label='Observed (USGS)', linewidth=1.2, alpha=0.7)
        axes[0, 0].plot(sim_cal.index, sim_cal.values, 'r-', label='Calibrated (SUMMA+MODFLOW)', linewidth=1.2, alpha=0.7)
        axes[0, 0].set_ylabel('Discharge (m\u00b3/s)')
        axes[0, 0].set_title('Calibrated Streamflow')
        axes[0, 0].legend()
        axes[0, 0].grid(True, alpha=0.3)
        axes[0, 0].text(0.02, 0.95, f"NSE: {cal_metrics['NSE']}\nKGE: {cal_metrics['KGE']}\nBias: {cal_metrics['PBIAS']}%",
                        transform=axes[0, 0].transAxes, verticalalignment='top',
                        bbox=dict(facecolor='white', alpha=0.8), fontsize=9)

        # Scatter
        axes[0, 1].scatter(obs_cal, sim_cal, alpha=0.5, s=10)
        max_val = max(obs_cal.max(), sim_cal.max())
        axes[0, 1].plot([0, max_val], [0, max_val], 'k--', alpha=0.5)
        axes[0, 1].set_xlabel('Observed (m\u00b3/s)')
        axes[0, 1].set_ylabel('Simulated (m\u00b3/s)')
        axes[0, 1].set_title('Observed vs Simulated (Calibrated)')
        axes[0, 1].grid(True, alpha=0.3)

        # Monthly climatology
        monthly_obs_c = obs_cal.groupby(obs_cal.index.month).mean()
        monthly_sim_c = sim_cal.groupby(sim_cal.index.month).mean()
        axes[1, 0].plot(monthly_obs_c.index, monthly_obs_c.values, 'b-o', label='Observed', markersize=6)
        axes[1, 0].plot(monthly_sim_c.index, monthly_sim_c.values, 'r-o', label='Calibrated', markersize=6)
        axes[1, 0].set_xticks(range(1, 13))
        axes[1, 0].set_xticklabels(month_names)
        axes[1, 0].set_ylabel('Mean Discharge (m\u00b3/s)')
        axes[1, 0].set_title('Seasonal Flow Regime (Calibrated)')
        axes[1, 0].legend()
        axes[1, 0].grid(True, alpha=0.3)

        # Flow duration curve
        obs_s = obs_cal.sort_values(ascending=False)
        sim_s = sim_cal.sort_values(ascending=False)
        obs_r = np.arange(1., len(obs_s) + 1) / len(obs_s) * 100
        sim_r = np.arange(1., len(sim_s) + 1) / len(sim_s) * 100
        axes[1, 1].semilogy(obs_r, obs_s, 'b-', label='Observed', linewidth=2)
        axes[1, 1].semilogy(sim_r, sim_s, 'r-', label='Calibrated', linewidth=2)
        axes[1, 1].set_xlabel('Exceedance Probability (%)')
        axes[1, 1].set_ylabel('Discharge (m\u00b3/s)')
        axes[1, 1].set_title('Flow Duration Curve (Calibrated)')
        axes[1, 1].legend()
        axes[1, 1].grid(True, alpha=0.3)

        plt.suptitle('Sagehen Creek \u2014 Calibrated SUMMA+MODFLOW Evaluation', fontsize=14, fontweight='bold')
        plt.tight_layout()
        plt.show()
    else:
        print(f"Calibrated output directories not found at expected locations.")
        print(f"  Checked: {cal_summa_dir}")
        print(f"  Checked: {cal_modflow_dir}")
        cal_metrics = uncal_metrics
else:
    print("No calibration output directories found.")
    cal_metrics = uncal_metrics

### Step 5d — Flow Decomposition Analysis

Decompose total simulated streamflow into MODFLOW groundwater baseflow and SUMMA surface runoff components, and compare against observed USGS streamflow.

In [ ]:
# Step 5d — Flow Decomposition

import json, numpy as np, pandas as pd, matplotlib.pyplot as plt
import matplotlib.dates as mdates
from symfluence.models.modflow.coupling import SUMMAToMODFLOWCoupler
from symfluence.models.modflow.extractor import MODFLOWResultExtractor

# --- Data extraction (same as before) ---
opt_dir = project_dir / 'optimization' / 'COUPLED_GW'
exp_id = config.domain.experiment_id
best_run = opt_dir / f'dds_{exp_id}'
if not best_run.exists():
    run_dirs = sorted(opt_dir.glob('dds_workshop_run_*')) if opt_dir.exists() else []
    best_run = run_dirs[-1] if run_dirs else None

final_eval_dir = best_run / 'final_evaluation' if best_run else None
if final_eval_dir and final_eval_dir.exists():
    summa_dir, modflow_dir = final_eval_dir / 'SUMMA', final_eval_dir / 'MODFLOW'
else:
    summa_dir = project_dir / 'simulations' / exp_id / 'SUMMA'
    modflow_dir = project_dir / 'simulations' / exp_id / 'MODFLOW'

config_dict = config.to_dict(flatten=True)
coupler = SUMMAToMODFLOWCoupler(config_dict)
extractor = MODFLOWResultExtractor()

fast_runoff = coupler.extract_fast_runoff(summa_dir)
drain_discharge = extractor.extract_variable(
    modflow_dir, 'drain_discharge',
    start_date=str(config.domain.time_start)[:10], stress_period_length=1.0)

import xarray as xr
with xr.open_dataset(project_dir / 'settings' / 'SUMMA' / 'attributes.nc') as ds:
    A = float(ds['HRUarea'].values.sum())

total_flow = coupler.combine_flows(fast_runoff, drain_discharge, A)
surf = (fast_runoff * A).copy(); surf.index = pd.to_datetime(surf.index)
base = (drain_discharge.abs() / 86400.0).copy(); base.index = pd.to_datetime(base.index)

obs_csv = sorted((project_dir / 'observations' / 'streamflow' / 'preprocessed').glob('*.csv'))[0]
obs_daily = pd.read_csv(obs_csv, parse_dates=['datetime']).set_index('datetime')['discharge_cms'].resample('D').mean().dropna()

t0 = pd.Timestamp('2017-01-01')
cal_end = pd.Timestamp(config.domain.calibration_period.split(',')[1].strip())
ci = total_flow.index.intersection(obs_daily.index)
ci = ci[ci >= t0]

obs, sim = obs_daily.loc[ci], total_flow.reindex(ci).fillna(0)
bf, sf = base.reindex(ci).fillna(0), surf.reindex(ci).fillna(0)

def kge(o, s):
    r = np.corrcoef(o, s)[0,1]; return 1 - np.sqrt((r-1)**2 + (np.std(s)/np.std(o)-1)**2 + (np.mean(s)/np.mean(o)-1)**2)

# ============================================================================
# Minimal 2-panel figure
# ============================================================================
plt.rcParams.update({'font.size': 10, 'axes.spines.top': False, 'axes.spines.right': False})
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 7), height_ratios=[3, 1.2],
                                sharex=False, gridspec_kw={'hspace': 0.35})

# --- Top: smoothed hydrograph ---
w = 7
bf_s, sf_s, obs_s = [s.rolling(w, center=True, min_periods=1).mean() for s in (bf, sf, obs)]

ax1.fill_between(ci, 0, bf_s, color='#4A90D9', alpha=0.5, lw=0)
ax1.fill_between(ci, bf_s, bf_s + sf_s, color='#F5A623', alpha=0.5, lw=0)
ax1.plot(ci, obs_s, color='#333', lw=1.0, alpha=0.9)

# Eval period shading

ax1.set_ylabel('Q (m\u00b3/s)')
ax1.set_ylim(bottom=0)
ax1.xaxis.set_major_locator(mdates.YearLocator())
ax1.xaxis.set_major_formatter(mdates.DateFormatter('%Y'))
ax1.tick_params(axis='x', length=0)

# Minimal legend
from matplotlib.patches import Patch
from matplotlib.lines import Line2D
handles = [
    Patch(fc='#4A90D9', alpha=0.5, label='Baseflow'),
    Patch(fc='#F5A623', alpha=0.5, label='Surface'),
    Line2D([0],[0], color='#333', lw=1, label='Observed'),
]
ax1.legend(handles=handles, loc='upper right', frameon=False, fontsize=9, ncol=3)

# KGE annotation
ax1.text(0.01, 0.96, f'KGE = {kge(obs.values, sim.values):.3f}',
         transform=ax1.transAxes, va='top', fontsize=11, fontweight='bold', color='#333')

# Period labels

# --- Bottom: monthly stacked bars ---
mn = ['J','F','M','A','M','J','J','A','S','O','N','D']
mo = np.arange(1, 13)
m_bf = bf.groupby(bf.index.month).mean().reindex(mo).fillna(0).values
m_sf = sf.groupby(sf.index.month).mean().reindex(mo).fillna(0).values
m_obs = obs.groupby(obs.index.month).mean().reindex(mo).fillna(0).values
m_bfi = np.where(m_bf + m_sf > 0, m_bf / (m_bf + m_sf) * 100, 0)

x = np.arange(12)
ax2.bar(x, m_bf, 0.55, color='#4A90D9', alpha=0.65, lw=0)
ax2.bar(x, m_sf, 0.55, bottom=m_bf, color='#F5A623', alpha=0.65, lw=0)
ax2.scatter(x, m_obs, s=28, color='#333', zorder=5, marker='o')

# BFI on secondary axis
ax2b = ax2.twinx()
ax2b.plot(x, m_bfi, color='navy', lw=1.2, ls='--', alpha=0.6, marker='.', ms=4)
ax2b.set_ylabel('BFI %', fontsize=9, color='navy')
ax2b.set_ylim(0, 105)
ax2b.tick_params(axis='y', colors='navy', labelsize=8)
ax2b.spines['right'].set_visible(True)
ax2b.spines['right'].set_color('navy')
ax2b.spines['right'].set_alpha(0.3)
ax2b.spines['top'].set_visible(False)

ax2.set_xticks(x)
ax2.set_xticklabels(mn, fontsize=9)
ax2.set_ylabel('Mean Q (m\u00b3/s)')
ax2.set_xlim(-0.6, 11.6)
ax2.tick_params(axis='x', length=0)

fig.suptitle('Sagehen Creek — Flow Decomposition', fontsize=13, fontweight='bold', y=0.98)

plots_dir = project_dir / 'plots'
plots_dir.mkdir(exist_ok=True)
plt.savefig(str(plots_dir / 'flow_decomposition.png'), dpi=200, bbox_inches='tight',
            facecolor='white', edgecolor='none')
plt.show()

# Summary
mean_bfi = np.nanmean(m_bfi[m_bfi > 0])
print(f'Baseflow: {bf.mean():.3f} m\u00b3/s ({bf.mean()/(bf.mean()+sf.mean())*100:.0f}%)  |  '
      f'Surface: {sf.mean():.3f} m\u00b3/s ({sf.mean()/(bf.mean()+sf.mean())*100:.0f}%)  |  '
      f'BFI: {mean_bfi:.0f}%')